# Code Matlab

In [1]:
# clear;
# clc;
# format long e    % çà ce n'est pas intelligent, la constante d'Euler c'est e dans Matlab: change de nom!

# L = 1.0;  % longueur du lit
# ug = 0.01; % vitesse spécifique de gaz
# epsb = 0.5; % porosité du lit
# kg = 0.0001; % coefficient de transfert
# Ke = 10;  % coeff d'équilibre
# dp = 0.005;   % diamètre de particule
# as = 6*(1-epsb)/dp;  % surface spécifique, dépend du diamètre de la sphère
# tend = 1000;  % fin du temps de simulation

# % il n'y a pas de dispersion axiale

# N = 100;
# M = 5;
# x = linspace(0, L, N+1);   % sépare le tube en N+1 compartiments de même longueur
# t = linspace(0, tend, M+1);  % sépare le temps en 6 moments de même durée; çà me semble bien court pour une dynamique...

# figure, hold on
# title('Analytical solution:')
# xlabel('x/L [-]');
# h = legend('show','location','best');
# set(h,'FontSize',12);
# for j=2:M+1
#     for i=1:N+1
#         tau = kg*as/((1-epsb)*Ke)*(t(j) - x(i)/ug);  % donne le \tau de la correction pour chaque x et L
#         xi = kg*as/(epsb*ug)*x(i);  % pareil avec le \xi
#         fnc = @(u) exp(-u).*besseli(0, sqrt(4*tau*u), 1)./exp(-abs(real(sqrt(4*tau*u))));  % scales besseli function to avoid overflow
#         I = integral(fnc, 0 , xi); % integral in the solution
#         e = exp(-tau); % no comment
#         g = besseli(0, sqrt(4*tau*xi))*exp(-xi); % la deuxième partie dans le Cs
#         Cg(i) = real(1 - e*I);   % donne le Cg, normalisé par Cg_in
#         Cs(i) = real(Ke*(1 - e*(I + g))); % pareil pour Cs
#     end
#     Cg(isinf(Cg)) = 0;
#     Cs(isinf(Cs)) = 0;

#     subplot(2,1,1), hold on
#     yyaxis left, hold on
#     plot(x/L,Cg,'-x','LineWidth',1,'MarkerSize',1,'DisplayName',num2str(t(j)))
#     ylabel('Cg/Cgin [-]');
#     ylim([0 1]);

#     yyaxis right, hold on
#     subplot(2,1,2), hold on
#     plot(x/L,Cs,'-x','LineWidth',1,'MarkerSize',1,'DisplayName',num2str(t(j)))
#     ylabel('Cs/Cgin [-]');
#     ylim([0 Ke]);
# end

# Attempt - PINN in python

## Chargement package

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# !pip install tensorflow
# !pip install deepxde

In [4]:
import numpy as np
from deepxde.backend import tf
import deepxde as dde
import matplotlib.pyplot as plt

No backend selected.
Finding available backend...


Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.
Instructions for updating:
non-resource variables are not supported in the long term


Found tensorflow.compat.v1
Setting the default backend to "tensorflow.compat.v1". You can change it in the ~/.deepxde/config.json file or export the DDE_BACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)


In [5]:
L = 1.0;            # longueur du lit
ug = 0.01;          # vitesse spécifique de gaz
epsb = 0.5;         # porosité du lit
kg = 0.0001;        # coefficient de transfert
Ke = 10;            # coeff d'équilibre
dp = 0.005;         # diamètre de particule
as_ = 6*(1-epsb)/dp; # surface spécifique, dépend du diamètre de la sphère
tend = 1000;        # fin du temps de simulation

## Partial Differential Equations

In [6]:
def pde(z, u):
    cs, cg = u[:, 0:1], u[:, 1:2]

    dcg_z = dde.grad.jacobian(u, z, i=0, j=0)
    dcg_t = dde.grad.jacobian(u, z, i=0, j=1)
    dcs_t = dde.grad.jacobian(u, z, i=1, j=1)

    eq1 = (dcg_t + ug * dcg_z) + (kg*as_/epsb) * (cg-cs/Ke)
    eq2 = dcs_t - (kg*as_/(1-epsb)) * (cg-cs/Ke)

    return [eq1, eq2]

In [7]:
def cs_func(x):
  return 0

def cg_func(x):
  return 0

## Spécification des dimensions (z,t)

In [8]:
# Pas de calcul (0.1) ?
geom = dde.geometry.Interval(0, L)
timedomain = dde.geometry.TimeDomain(0, 0.99)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

## Initial Condition & Boundary Condition

### IC

In [10]:
ic_cs = dde.icbc.IC(geomtime, lambda cs: 0, lambda _, on_initial: on_initial, component=0)
ic_cg = dde.icbc.IC(geomtime, lambda cg: 1, lambda _, on_initial: on_initial, component=1)

### BC

In [11]:
bc_cs = dde.icbc.DirichletBC(geomtime, cs_func, lambda _, on_boundary: on_boundary, component=0)
bc_cg = dde.icbc.DirichletBC(geomtime, cg_func, lambda _, on_boundary: on_boundary, component=1)

## Combinaison de tous les paramètres définis

In [12]:
data = dde.data.TimePDE(
    geomtime,
    pde,
    [bc_cs, bc_cg, ic_cs, ic_cg],
    num_domain=2540,
    num_boundary=80,
    num_initial=160
)

## Construction réseaux de neurones

In [13]:
layer_size = [2] + [50] * 4 + [1]
activation = "tanh"
initializer = "Glorot normal"

net = dde.nn.FNN(layer_size, activation, initializer)

## Apprentissage du modèle

In [15]:
model = dde.Model(data, net)

model.compile("adam", lr=1e-3)
losshistory, train_state = model.train(iterations=15000)
model.compile("L-BFGS")
losshistory, train_state = model.train()

Compiling model...


ValueError: ignored

## Affichage des résultats

In [ ]:
x = np.linspace(0, L, num = L*100)
x = np.reshape(x, (-1, 1))
t = np.linspace(0, 1, num = 100)
t = np.reshape(t, (-1, 1))

color = ['g', 'y', 'c', 'b', 'm', 'r', 'k']
line = ['-', '--', '-.']
line_idx = 0

T = [0, 24, 50, 90]

plt.figure(figsize=(20,12))
color_idx = 0
for idx_time in T:
  x_t = [[0,0]]
  x_t = np.append(x_t, np.insert(x, 1, t[idx_time][0], axis=1), axis=0)
  x_t = x_t[1:]

  plt.plot(
      x_t[:,0],
      model.predict(x_t).flatten(),
      color[color_idx % 7],
      linestyle = line[line_idx],
      label = "at t= " + str(t[idx_time])
  )
  color_idx +=1
line_idx +=1

color_idx = 0

plt.ylabel("u (m/s)")
plt.xlabel('position x (m)')
plt.legend()

## Sauvegarde

In [ ]:
dde.saveplot(losshistory, train_state, issave=True, isplot=True)